# Sagemaker Experiment Logger for PyTorch-Lightning
 

In [ ]:
import os

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from sagemaker.experiments.run import Run

from example.model import MNISTModel
from experiments_addon.logger import SagemakerExperimentsLogger

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_model = MNISTModel(data_dir=PATH_DATASETS)

## Use SagemakerExperimentsLogger by explicitly passing in run_name and experiment_name

In [ ]:
sagemaker_logger = SagemakerExperimentsLogger(
    experiment_name="TestExp",
    run_name="TestRun"
)

trainer = Trainer(
    logger=sagemaker_logger,
    accelerator="auto",
    devices=1,
    max_epochs=3,
)

## Use the SagemakerExperimentsLogger in a notebook within a run context

In [ ]:
with Run(experiment_name="testExperiment", run_name="testRun1") as _:
    logger = SagemakerExperimentsLogger()
    trainer = Trainer(logger=logger)

## Use the SagemakerExperimentsLogger in a notebook with other logger

In [ ]:
tensorboard_logger = TensorBoardLogger()
with Run(experiment_name="testExperiment", run_name="testRun2") as _:
    logger = SagemakerExperimentsLogger()
    trainer = Trainer(logger=[logger, tensorboard_logger])